In [1]:
# settings
from dotenv import load_dotenv
load_dotenv('semantic-kernel/python/.env')
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = "gpt-4-1106-preview"

# OpenAI SDK
* this requires opanai >= 1.0.0. Be sure to upgrade the package but by doing so, old code won't work any more.

In [2]:
# custom functions
import datetime
def current_date(args:dict):
    return datetime.datetime.now().strftime("%Y-%m-%d")

# mock API call
def get_order_status(args:dict):
    if args['order_id'] == '1':
        order = {'item_name':'A', 'order_date':'2023-10-10', 'delivery_status':'completed', 'delivery_date':'2023-10-11'}
    if args['order_id'] == '2':
        order = {'item_name':'B', 'order_date':'2023-11-22', 'delivery_status':'completed', 'delivery_date':'2023-11-23'}
    if args['order_id'] == '3':
        order = {'item_name':'C', 'order_date':'2023-12-01', 'delivery_status':'in delivery', 'delivery_date':'2023-12-15'}
    return str(order)

In [5]:
# create instance with tools
from openai import OpenAI
import json
import time
client = OpenAI()
assistant = client.beta.assistants.create(
    name="EC assistant",
    instructions="You answer questions from EC customer about their orders and returns",
    tools=[{
      "type": "function",
                "function": {
                    "name": "get_order_status",
                    "description": "Get order delivery status and date",
                    "parameters": {
                        "type": "object",
                        "properties": {
                        "order_id": {"type": "string", "description": "ID to identify the order"}
                        },
                    "required": ["order_id"],
                    }
                }
    }, {
        "type": "function", 
        "function": {"name": "current_date", 
            "description": "Get today's date", 
            "parameters": {'type': "object", 'properties': {}, 'required': [],}}
    }],
model= model
)

In [20]:
# create thread and add messages
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the status of my order for B? order_id is 2." 
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. Return policy : accept returns within 30 days from the delivery date with full refund without asking reason."
)

In [19]:
# get response but it stops at the first function call function need to excecuted separately and the result should be added to the thread.
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
    )
print(run.status)

requires_action


In [9]:
# API response is quite complicated object. You need to parse it to get id, name of function to run, and arguments to pass to the function.
print(run)

Run(id='run_hf7FcCxxdKQTx8FDOSlYi92b', assistant_id='asst_VuLXzH2SDIJAaqVrhRxRWBaO', cancelled_at=None, completed_at=None, created_at=1701454796, expires_at=1701455396, failed_at=None, file_ids=[], instructions='Please address the user as Jane Doe. Return policy : accept returns within 30 days from the delivery date with full refund without asking reason.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1701454797, status='in_progress', thread_id='thread_IP7GEJZ4S95XSjMeO105qZ40', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='get_order_status', parameters={'type': 'object', 'properties': {'order_id': {'type': 'string', 'description': 'ID to identify the order'}}, 'required': ['order_id']}, description='Get order delivery status and date'), type='function'), ToolAssistantToolsFunction(function=FunctionDefinition(name='current_date', parameters={'type': 'object', 'properties': {}, 'required': []}, desc

In [21]:
while run.status in ['queued','in_progress','requires_action']:
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )
    print(run.status)

    if run.status == 'requires_action':
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        ids = []
        function_response = []
        for i in tool_calls:
            ids.append(i.id)
            function_name = i.function.name
            function_arguments = json.loads(i.function.arguments)
            function_response.append(globals()[function_name](function_arguments))
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=[{'tool_call_id':i, 'output':r}  for i,r in zip(ids,function_response)]
        )
    time.sleep(3)

requires_action
in_progress
completed


In [12]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

for i in reversed(messages.data):
    print(i.role, ':', i.content[0].text.value)

user : What is the status of my order for B? order_id is 2.
assistant : The status of your order for the item "B" with order_id "2" is completed. It was delivered on November 23, 2023. If you have any further questions or concerns, please let me know, Jane Doe.


In [13]:
steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

for i in steps.data:
    print(i.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_6Qt1nTlkVafRhk5XAXBoKuXi'), type='message_creation')
ToolCallsStepDetails(tool_calls=[FunctionToolCall(id='call_MIpCkZm87R3Op7EmrsNGTnLU', function=Function(arguments='{"order_id":"2"}', name='get_order_status', output="{'item_name': 'B', 'order_date': '2023-11-22', 'delivery_status': 'completed', 'delivery_date': '2023-11-23'}"), type='function')], type='tool_calls')


In [22]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can I return this item?" 
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. Return policy : accept returns within 30 days from the delivery date with full refund without asking reason."
)

In [23]:
while run.status in ['queued','in_progress','requires_action']:
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )
    print(run.status)

    if run.status == 'requires_action':
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        ids = []
        function_response = []
        for i in tool_calls:
            ids.append(i.id)
            function_name = i.function.name
            function_arguments = json.loads(i.function.arguments)
            function_response.append(globals()[function_name](function_arguments))
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=[{'tool_call_id':i, 'output':r}  for i,r in zip(ids,function_response)]
        )
    time.sleep(3)

requires_action
in_progress
in_progress
completed


In [24]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

for i in reversed(messages.data):
    print(i.role, ':', i.content[0].text.value)

user : What is the status of my order for B? order_id is 2.
assistant : The status of your order for item B with order ID 2 is that it has been completed. The item was delivered on November 23, 2023. If you have any further inquiries or need assistance with this order, please let me know.
user : Can I return this item?
assistant : As of the current date, December 1, 2023, you are still within the 30-day return window that begins from the delivery date of November 23, 2023. Therefore, you can return the item for a full refund without needing to provide a reason for the return.

If you would like to proceed with the return or need instructions on how to do so, please let me know, and I'll be happy to assist you further.
